---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
pd.DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], columns=["State", "RegionName"]  )

,State,RegionName
0,Michigan,Ann Arbor
1,Michigan,Yipsilanti


In [164]:
with open("university_towns.txt") as file:
    temp_list = []
    file.readline()
    for line in file:
        temp_list.append(line[:-1])
        
df_dict = {}
state_list = []
#df_dict["State"] = []
#df_dict["RegionName"] = []

for item in temp_list:
    item = item.replace('â€“','-')
    if item in states.values():
        state_list.append(item)

for item in temp_list:
    if item in state_list:
        state = item
        df_dict[state] = []
        continue
        
    if "[" in item and "(" in item:
        idx_1 = item.index("[")
        idx_2 = item.index("(")-1
        idx_first = min(idx_1, idx_2)
        new_item = item[:idx_first]
        
        df_dict[state].append(new_item)
                
    elif "(" in item:
        new_item1 = item[:item.index("(")-1]
        df_dict[state].append(new_item1)
        
    elif "[" in item:
        new_item1 = item[:item.index("[")]
        df_dict[state].append(new_item1)
        
    elif ":" in item:
        item = item.strip(":")
    else:
        df_dict[state].append(item)
        
   
    

Alabama <class 'str'>
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
bu da state: Alabama
Alaska <class 'str'>
bu da state: Alaska
bu da state: Alaska
Arizona <class 'str'>
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
bu da state: Arizona
Arkansas <class 'str'>
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
bu da state: Arkansas
California <class 'str'>
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu da state: California
bu d

In [57]:
df = pd.DataFrame.from_dict(df_dict, orient='index')
df = df.T
# veya
df1 = pd.DataFrame({ key:pd.Series(value) for key, value in df_dict.items() })

In [64]:
final_list = []
for key, value in df_dict.items():
    for i in range(len(value)):
        final_list.append([key,df_dict[key][i]])
    
final_list

[['Alabama', 'Auburn'],
 ['Alabama', 'Birmingham'],
 ['Alabama', 'Dothan'],
 ['Alabama', 'Florence'],
 ['Alabama', 'Homewood'],
 ['Alabama', 'Huntsville'],
 ['Alabama', 'Jacksonville'],
 ['Alabama', 'Livingston'],
 ['Alabama', 'Mobile'],
 ['Alabama', 'Montevallo'],
 ['Alabama', 'Montgomery'],
 ['Alabama', 'Troy'],
 ['Alabama', 'Tuscaloosa'],
 ['Alabama', 'Tuskegee'],
 ['Alaska', 'Anchorage'],
 ['Alaska', 'Fairbanks'],
 ['Alaska', 'Juneau'],
 ['Alaska', 'Ketchikan'],
 ['Alaska', 'Sitka'],
 ['Arizona', 'Chandler'],
 ['Arizona', 'Flagstaff'],
 ['Arizona', 'Gilbert'],
 ['Arizona', 'Glendale'],
 ['Arizona', 'Lake Havasu City'],
 ['Arizona', 'Mesa'],
 ['Arizona', 'Peoria'],
 ['Arizona', 'Phoenix'],
 ['Arizona', 'Prescott'],
 ['Arizona', 'Scottsdale'],
 ['Arizona', 'Tempe'],
 ['Arizona', 'Tucson'],
 ['Arkansas', 'Arkadelphia'],
 ['Arkansas', 'Conway'],
 ['Arkansas', 'Fayetteville'],
 ['Arkansas', 'Fort Smith'],
 ['Arkansas', 'Jonesboro'],
 ['Arkansas', 'Little Rock'],
 ['Arkansas', 'Magnolia'

In [65]:
df_v = pd.DataFrame(final_list, columns=["State", "RegionName"])
df_v

,State,RegionName
0,Alabama,Auburn
1,Alabama,Birmingham
2,Alabama,Dothan
3,Alabama,Florence
4,Alabama,Homewood
5,Alabama,Huntsville
6,Alabama,Jacksonville
7,Alabama,Livingston
8,Alabama,Mobile
9,Alabama,Montevallo


In [66]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    with open("university_towns.txt") as file:
        temp_list = []
        file.readline()
        for line in file:
            temp_list.append(line[:-1])

    df_dict = {}
    
    for item in temp_list:
        item = item.replace('â€“','-')
        if item in states.values():
            state = item
            df_dict[state] = []
            continue

        if "[" in item and "(" in item:
            idx_1 = item.index("[")
            idx_2 = item.index("(")-1
            idx_first = min(idx_1, idx_2)
            new_item = item[:idx_first]
            df_dict[state].append(new_item)

        elif "(" in item:
            new_item1 = item[:item.index("(")-1]
            df_dict[state].append(new_item1)

        elif "[" in item:
            new_item1 = item[:item.index("[")]
            df_dict[state].append(new_item1)

        elif ":" in item:
            item = item.strip(":")
        else:
            df_dict[state].append(item)

    final_list = []
    for key, value in df_dict.items():
        for i in range(len(value)):
            final_list.append([key,df_dict[key][i]])

    df = pd.DataFrame(final_list, columns=["State", "RegionName"])
                
    return df

In [87]:
gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

serie = gdp.Quarter.str.find("2000")

for idx, value in serie.iteritems():
    if value >= 0:
        q2000 = idx
        break
gdp = gdp.iloc[q2000:]

result = []
for i in range(len(gdp) - 2):
    if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
        result.append(gdp.iloc[i+1][0])
result[0]


'2008Q3'

In [88]:
result

['2008Q3', '2008Q4', '2009Q1']

In [90]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
    gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

    serie = gdp.Quarter.str.find("2000")

    for idx, value in serie.iteritems():
        if value >= 0:
            q2000 = idx
            break
    gdp = gdp.iloc[q2000:]

    result = []
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] > gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] > gdp.iloc[i+2][1]):
            result.append(gdp.iloc[i+1][0])
    
    return result[0]

In [97]:
rec_start = get_recession_start()
gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

first_idx = gdp[gdp.Quarter == rec_start].index.tolist()[0]

gdp = gdp.iloc[first_idx:]
for i in range(2, len(gdp)):
    if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
        print(gdp.iloc[i][0])
        break

2009Q4


In [99]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    rec_start = get_recession_start()
    gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
    gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

    first_idx = gdp[gdp.Quarter == rec_start].index.tolist()[0]

    gdp = gdp.iloc[first_idx:]
    for i in range(2, len(gdp)):
        if (gdp.iloc[i-2][1] < gdp.iloc[i-1][1]) and (gdp.iloc[i-1][1] < gdp.iloc[i][1]):
            return gdp.iloc[i][0]

In [100]:
get_recession_end()

'2009Q4'

In [101]:
rec_start = get_recession_start()
rec_end = get_recession_end()
gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

first_idx = gdp[gdp.Quarter == rec_start].index.tolist()[0]
last_index = gdp[gdp.Quarter == rec_end].index.tolist()[0]
    
gdp = gdp.iloc[first_idx : last_index + 1]
result = gdp['GDP'].min()
result_idx = gdp[gdp['GDP'] == result].index.tolist()[0] - first_idx
gdp.iloc[result_idx]['Quarter']

'2009Q2'

In [102]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    rec_start = get_recession_start()
    rec_end = get_recession_end()
    gdp = pd.read_excel('gdplev.xlsx', skiprows = 7, usecols= {'Unnamed: 4', 'Unnamed: 6'})
    gdp = gdp.rename(columns = {'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})

    first_idx = gdp[gdp.Quarter == rec_start].index.tolist()[0]
    last_index = gdp[gdp.Quarter == rec_end].index.tolist()[0]

    gdp = gdp.iloc[first_idx : last_index + 1]
    result = gdp['GDP'].min()
    result_idx = gdp[gdp['GDP'] == result].index.tolist()[0] - first_idx
    
    return gdp.iloc[result_idx]['Quarter']

In [176]:
hous_df = pd.read_csv('City_Zhvi_AllHomes.csv', encoding='latin-1')
#hous_df['State'] = hous_df['State'].map(states)
#hous_df = hous_df.set_index(["State","RegionName"])
#columns = hous_df.columns
#selected_columns = []
#for col in columns:
#    try:
#        if int(col[:4]) >= 2000 and int(col[:4]) < 2017:
#            selected_columns.append(col)
#    except:
#        continue
#hous_df = hous_df[selected_columns]
#hous_df = hous_df.groupby(pd.PeriodIndex(hous_df.columns, freq='Q'), axis=1).mean()
##df = df.iloc[:, :-1] bu komut ile de son sütün u düşürmüş oluyoruz.
#hous_df = hous_df.drop(hous_df.columns[len(hous_df.columns)-1], axis=1)
#hous_df.head()

In [106]:
hous_df = pd.read_csv('City_Zhvi_AllHomes.csv', encoding='latin-1')
#convert two-letter-state to full name of state
hous_df['State'] = hous_df['State'].map(states)
#set index to state, regionname
hous_df = hous_df.set_index(["State","RegionName"])
#filter columns by year, only want 2000 to 2016
hous_df = hous_df.filter(regex='^20', axis=1)
#group select columns by quarter, calculates average per quarter
hous_df = hous_df.groupby(pd.PeriodIndex(hous_df.columns, freq='Q'), axis=1).mean()
hous_df

,,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2017Q2,2017Q3,2017Q4,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,220433.333333,228866.666667,236166.666667,242266.666667,248633.333333,256000.000000,265700.000000,277100.000000,285033.333333,286300.000000,...,6.237333e+05,6.368000e+05,6.429667e+05,6.522333e+05,6.652667e+05,6.713000e+05,6.800333e+05,6.820667e+05,6.787333e+05,6.716667e+05
California,Los Angeles,208166.666667,214900.000000,221466.666667,227766.666667,234566.666667,241333.333333,248200.000000,255833.333333,265500.000000,277000.000000,...,6.233000e+05,6.343667e+05,6.465000e+05,6.632667e+05,6.727000e+05,6.785000e+05,6.879333e+05,6.934667e+05,6.916000e+05,6.940667e+05
Texas,Houston,106266.666667,106300.000000,106233.333333,106500.000000,106600.000000,107033.333333,108066.666667,109466.666667,110533.333333,111300.000000,...,1.673333e+05,1.683333e+05,1.712000e+05,1.744000e+05,1.778667e+05,1.807333e+05,1.831333e+05,1.869000e+05,1.866000e+05,1.894667e+05
Illinois,Chicago,142233.333333,147433.333333,153033.333333,158700.000000,163566.666667,167033.333333,171433.333333,176833.333333,182366.666667,187033.333333,...,2.200333e+05,2.214333e+05,2.233333e+05,2.268333e+05,2.279000e+05,2.275000e+05,2.282000e+05,2.303333e+05,2.296667e+05,2.276667e+05
Texas,San Antonio,92133.333333,91800.000000,91133.333333,90800.000000,90300.000000,89700.000000,89433.333333,89400.000000,89500.000000,90000.000000,...,1.542333e+05,1.568333e+05,1.593000e+05,1.638667e+05,1.673333e+05,1.701667e+05,1.737333e+05,1.764667e+05,1.768667e+05,1.786333e+05
Pennsylvania,Philadelphia,51466.666667,52500.000000,53600.000000,54433.333333,55200.000000,55966.666667,56500.000000,57300.000000,58800.000000,60366.666667,...,1.337333e+05,1.369000e+05,1.397333e+05,1.438333e+05,1.481333e+05,1.515667e+05,1.563000e+05,1.609667e+05,1.559000e+05,1.581333e+05
Arizona,Phoenix,114500.000000,116700.000000,118800.000000,120933.333333,123366.666667,125266.666667,126600.000000,127900.000000,129500.000000,131133.333333,...,2.104000e+05,2.154667e+05,2.198000e+05,2.254667e+05,2.303000e+05,2.342000e+05,2.391667e+05,2.429000e+05,2.432000e+05,2.448667e+05
Nevada,Las Vegas,136933.333333,138266.666667,139100.000000,140333.333333,142333.333333,144600.000000,147333.333333,150800.000000,154600.000000,157266.666667,...,2.237000e+05,2.306000e+05,2.381000e+05,2.475333e+05,2.552667e+05,2.652333e+05,2.740333e+05,2.777000e+05,2.768333e+05,2.743000e+05
California,San Diego,224066.666667,235866.666667,246833.333333,256633.333333,266900.000000,277166.666667,286233.333333,295700.000000,306866.666667,319900.000000,...,5.787333e+05,5.889000e+05,5.996000e+05,6.161333e+05,6.221667e+05,6.275000e+05,6.329333e+05,6.347333e+05,6.368333e+05,6.368667e+05


In [134]:
hous_df.shape

(13108, 67)

In [135]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    hous_df = pd.read_csv('City_Zhvi_AllHomes.csv', encoding='latin-1')
    hous_df['State'] = hous_df['State'].map(states)
    hous_df = hous_df.set_index(["State","RegionName"])
    columns = hous_df.columns
    selected_columns = []
    for col in columns:
        try:
            if int(col[:4]) >= 2000 and int(col[:4]) < 2017:
                selected_columns.append(col)
        except:
            continue
    hous_df = hous_df[selected_columns]
    hous_df = hous_df.groupby(pd.PeriodIndex(hous_df.columns, freq='Q'), axis=1).mean()
    hous_df = hous_df.drop(hous_df.columns[len(hous_df.columns)-1], axis=1)
    
    return hous_df

In [171]:
data = convert_housing_data_to_quarters().copy()
cols = data.columns.map(str)
#cols2 = data.columns.strftime('%YQ%m')
#cols = data.columns.to_list()
#cols = data.columns.to_list()
new_cols = []
for col in cols:
    col = str(col)
    new_cols.append(col)
cols

Index(['2000Q1', '2000Q2', '2000Q3', '2000Q4', '2001Q1', '2001Q2', '2001Q3',
       '2001Q4', '2002Q1', '2002Q2', '2002Q3', '2002Q4', '2003Q1', '2003Q2',
       '2003Q3', '2003Q4', '2004Q1', '2004Q2', '2004Q3', '2004Q4', '2005Q1',
       '2005Q2', '2005Q3', '2005Q4', '2006Q1', '2006Q2', '2006Q3', '2006Q4',
       '2007Q1', '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
       '2008Q4', '2009Q1', '2009Q2', '2009Q3', '2009Q4', '2010Q1', '2010Q2',
       '2010Q3', '2010Q4', '2011Q1', '2011Q2', '2011Q3', '2011Q4', '2012Q1',
       '2012Q2', '2012Q3', '2012Q4', '2013Q1', '2013Q2', '2013Q3', '2013Q4',
       '2014Q1', '2014Q2', '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3',
       '2015Q4', '2016Q1', '2016Q2', '2016Q3'],
      dtype='object')

In [159]:
df = convert_housing_data_to_quarters().copy()
cols = df.columns.to_list()
new_cols = []
for col in cols:
    col = str(col)
    new_cols.append(col)
df.columns = new_cols
df = df.loc[:,'2008Q3':'2009Q2']
df = df.reset_index()
def price_ratio(row):
    return (row['2008Q3'] - row['2009Q2'])/row['2008Q3']
    
df['up_down'] = df.apply(price_ratio,axis=1)
#uni data 
    
uni_town = get_list_of_university_towns()['RegionName']
uni_town = set(uni_town)

def is_uni_town(row):
    #check if the town is a university towns or not.
    if row['RegionName'] in uni_town:
        return 1
    else:
        return 0
df['is_uni'] = df.apply(is_uni_town,axis=1)
    
    
not_uni = df[df['is_uni']==0].loc[:,'up_down'].dropna()
is_uni  = df[df['is_uni']==1].loc[:,'up_down'].dropna()
def better():
    if not_uni.mean() < is_uni.mean():
        return 'non-university town'
    else:
        return 'university town'
p_val = list(ttest_ind(not_uni, is_uni))[1]
result = (True,p_val,better())
result

(True, 0.7046642292638823, 'university town')

In [160]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters().copy()
    cols = df.columns.to_list()
    new_cols = []
    for col in cols:
        col = str(col)
        new_cols.append(col)
    df.columns = new_cols
    df = df.loc[:,'2008Q3':'2009Q2']
    df = df.reset_index()
    def price_ratio(row):
        return (row['2008Q3'] - row['2009Q2'])/row['2008Q3']

    df['up_down'] = df.apply(price_ratio,axis=1)
    #uni data 

    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    def is_uni_town(row):
        #check if the town is a university towns or not.
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    df['is_uni'] = df.apply(is_uni_town,axis=1)


    not_uni = df[df['is_uni']==0].loc[:,'up_down'].dropna()
    is_uni  = df[df['is_uni']==1].loc[:,'up_down'].dropna()
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    result = (True,p_val,better())
    return result

run_ttest()

(True, 0.7046642292638823, 'university town')